In [5]:
def csp_train(train_data_1, train_data_2, numFilt):

    numTrials_1 = np.size(train_data_1, 0)
    numTrials_2 = np.size(train_data_2, 0)

    # train the CSP filters (generalized eigenvalue problem)
    csp_filts = calc_CSP(train_data_1, train_data_2, numFilt)

    # extract the features
    train_filt_1 = apply_CSP(train_data_1, csp_filts, numFilt)
    train_logP_1 = np.squeeze(np.log(np.var(train_filt_1, axis=2)))

    train_filt_2 = apply_CSP(train_data_2, csp_filts, numFilt)
    train_logP_2 = np.squeeze(np.log(np.var(train_filt_2, axis=2)))

    # define the classifier
    clf = lda.LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')

    # stack features for both classes
    X = np.concatenate((train_logP_1, train_logP_2), axis=0)
    y1 = np.zeros(numTrials_1)
    y2 = np.ones(numTrials_2)
    y = np.concatenate((y1, y2))

    # train the classifier
    clf.fit(X, y)

    return csp_filts, clf


def csp_test(test_data, csp_filts, clf):
    total_filts = csp_filts.shape[1]
    num_trials = test_data.shape[0]
    
    filtered_data = np.zeros((num_trials, total_filts))
    for trial in range(num_trials):
        trial_data = test_data[trial, :, :]  # Extract single trial (channels, samples)
        # Apply spatial filters
        trial_filtered = np.dot(csp_filts.T, trial_data)
        # Calculate log variance for the trial
        log_var = np.log(np.var(trial_filtered, axis=1))
        filtered_data[trial, :] = log_var
        
    # Classify using clf, get probability of class 0
    return clf.predict_proba(filtered_data)[:, 0]



def calc_CSP(x1, x2, numFilt):
    """
    Standard CSP using generalized eigenvalue decomposition.
    """
    num_trials_1 = np.size(x1, 0)
    num_trials_2 = np.size(x2, 0)
    n_chans = np.size(x1, 1)

    # Compute average trace-normalized covariance for class 1
    c1 = np.zeros((n_chans, n_chans))
    for i in range(num_trials_1):
        cov_i = np.cov(x1[i])
        cov_i /= (np.trace(cov_i) + 1e-12)
        c1 += cov_i
    c1 /= num_trials_1

    # Compute average covariance for class 2
    c2 = np.zeros((n_chans, n_chans))
    for i in range(num_trials_2):
        cov_i = np.cov(x2[i])
        cov_i /= (np.trace(cov_i) + 1e-12)
        c2 += cov_i
    c2 /= num_trials_2

    # Solve the generalized eigenvalue problem
    R = c1 + c2
    eigvals, eigvecs = sp.linalg.eigh(c1, R)

    # Sort eigenvectors by eigenvalues in descending order
    idx = np.argsort(eigvals)[::-1]
    eigvecs = eigvecs[:, idx]

    # select filters
    filts_1 = eigvecs[:, :numFilt]
    filts_1 = select_filts(filts_1, numFilt)

    filts_2 = eigvecs[:, -numFilt:]
    filts_2 = select_filts(filts_2, numFilt)

    return np.concatenate((filts_1, filts_2), axis=1)


def select_filts(filt, col_num):
    temp = np.shape(filt)
    columns = np.arange(0, col_num)
    f = filt[:, columns].copy()
    for ij in range(col_num):
        f[:, ij] /= np.linalg.norm(f[:, ij]) + 1e-12
    return f


def apply_CSP(X, f, col_num):
    f = np.transpose(f)
    temp = np.shape(X)
    num_trials = temp[0]

    dat = np.zeros((num_trials, 2*col_num, temp[2]))
    for ik in range(num_trials):
        dat[ik, :, :] = np.matmul(f, X[ik, :, :])
    return dat

## Left vs Rest

In [ ]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Declare useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
csp_per_class = 3

# Initialize accuracy and kappa arrays
accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        print(f'Processing Fold {fold+1}/{k}')

        # Initialize band scores separately for CCACSP and CSP
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))

        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data
            L_train = data_left[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_left[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            L_val = data_left[subject, band][:,:,valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_left[subject, band][:,:,valIdx[fold]].transpose(2,0,1)

            train_data = np.stack((L_train, OVR_train), axis=0)
            val_data = np.stack((L_val, OVR_val), axis=0)

            # CCACSP
            ccacsp_filts, clf_CCACSP = train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf_CCACSP)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf_CCACSP)

            band_score_train_CCACSP[:, band] = train_prob_CCACSP
            band_score_val_CCACSP[:, band] = val_prob_CCACSP

            # CSP 
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf_CSP)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf_CSP)


            band_score_train_CSP[:, band] = train_prob_CSP
            band_score_val_CSP[:, band] = val_prob_CSP


        # True labels
        y_train = np.append(np.ones(train_size), np.zeros(train_size))
        y_val = np.append(np.ones(val_size), np.zeros(val_size))

        # CCACSP metrics
        y_pred_train_CCACSP = np.rint(band_score_train_CCACSP.mean(1))
        y_pred_val_CCACSP = np.rint(band_score_val_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = sum(y_pred_val_CCACSP == y_val)/len(y_val)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_pred_val_CCACSP, y_val)

        # CSP metrics
        y_pred_train_CSP = np.rint(band_score_train_CSP.mean(1))
        y_pred_val_CSP = np.rint(band_score_val_CSP.mean(1))
        accuracy_CSP[subject, fold] = sum(y_pred_val_CSP == y_val)/len(y_val)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_pred_val_CSP, y_val)

    print('===============')

# Final Output
print("CCACSP mean accuracy per subject:", np.mean(accuracy_CCACSP, axis=1))
print("CCACSP mean kappa per subject:", np.mean(kappa_CCACSP, axis=1))
print("CSP mean accuracy per subject:", np.mean(accuracy_CSP, axis=1))
print("CSP mean kappa per subject:", np.mean(kappa_CSP, axis=1))

## Right vs Rest

In [ ]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Declare useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
csp_per_class = 3

# Initialize accuracy and kappa arrays
accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        print(f'Processing Fold {fold+1}/{k}')

        # Initialize band scores separately for CCACSP and CSP
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))

        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data
            r_train = data_right[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_right[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            r_val = data_right[subject, band][:,:,valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_right[subject, band][:,:,valIdx[fold]].transpose(2,0,1)

            train_data = np.stack((r_train, OVR_train), axis=0)
            val_data = np.stack((r_val, OVR_val), axis=0)

            # CCACSP
            ccacsp_filts, clf_CCACSP = train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf_CCACSP)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf_CCACSP)

            band_score_train_CCACSP[:, band] = train_prob_CCACSP
            band_score_val_CCACSP[:, band] = val_prob_CCACSP

            # CSP 
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf_CSP)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf_CSP)


            band_score_train_CSP[:, band] = train_prob_CSP
            band_score_val_CSP[:, band] = val_prob_CSP


        # True labels
        y_train = np.append(np.ones(train_size), np.zeros(train_size))
        y_val = np.append(np.ones(val_size), np.zeros(val_size))

        # CCACSP metrics
        y_pred_train_CCACSP = np.rint(band_score_train_CCACSP.mean(1))
        y_pred_val_CCACSP = np.rint(band_score_val_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = sum(y_pred_val_CCACSP == y_val)/len(y_val)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_pred_val_CCACSP, y_val)

        # CSP metrics
        y_pred_train_CSP = np.rint(band_score_train_CSP.mean(1))
        y_pred_val_CSP = np.rint(band_score_val_CSP.mean(1))
        accuracy_CSP[subject, fold] = sum(y_pred_val_CSP == y_val)/len(y_val)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_pred_val_CSP, y_val)

    print('===============')

# Final Output
print("CCACSP mean accuracy per subject:", np.mean(accuracy_CCACSP, axis=1))
print("CCACSP mean kappa per subject:", np.mean(kappa_CCACSP, axis=1))
print("CSP mean accuracy per subject:", np.mean(accuracy_CSP, axis=1))
print("CSP mean kappa per subject:", np.mean(kappa_CSP, axis=1))

## Feet vs Rest

In [ ]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Declare useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
csp_per_class = 3

# Initialize accuracy and kappa arrays
accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        print(f'Processing Fold {fold+1}/{k}')

        # Initialize band scores separately for CCACSP and CSP
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))

        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data
            f_train = data_feet[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_feet[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            f_val = data_feet[subject, band][:,:,valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_feet[subject, band][:,:,valIdx[fold]].transpose(2,0,1)

            train_data = np.stack((f_train, OVR_train), axis=0)
            val_data = np.stack((f_val, OVR_val), axis=0)

            # CCACSP
            ccacsp_filts, clf_CCACSP = train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf_CCACSP)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf_CCACSP)

            band_score_train_CCACSP[:, band] = train_prob_CCACSP
            band_score_val_CCACSP[:, band] = val_prob_CCACSP

            # CSP 
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf_CSP)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf_CSP)


            band_score_train_CSP[:, band] = train_prob_CSP
            band_score_val_CSP[:, band] = val_prob_CSP


        # True labels
        y_train = np.append(np.ones(train_size), np.zeros(train_size))
        y_val = np.append(np.ones(val_size), np.zeros(val_size))

        # CCACSP metrics
        y_pred_train_CCACSP = np.rint(band_score_train_CCACSP.mean(1))
        y_pred_val_CCACSP = np.rint(band_score_val_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = sum(y_pred_val_CCACSP == y_val)/len(y_val)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_pred_val_CCACSP, y_val)

        # CSP metrics
        y_pred_train_CSP = np.rint(band_score_train_CSP.mean(1))
        y_pred_val_CSP = np.rint(band_score_val_CSP.mean(1))
        accuracy_CSP[subject, fold] = sum(y_pred_val_CSP == y_val)/len(y_val)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_pred_val_CSP, y_val)

    print('===============')

# Final Output
print("CCACSP mean accuracy per subject:", np.mean(accuracy_CCACSP, axis=1))
print("CCACSP mean kappa per subject:", np.mean(kappa_CCACSP, axis=1))
print("CSP mean accuracy per subject:", np.mean(accuracy_CSP, axis=1))
print("CSP mean kappa per subject:", np.mean(kappa_CSP, axis=1))

## Tongue vs Rest

In [ ]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Declare useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
csp_per_class = 3

# Initialize accuracy and kappa arrays
accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        print(f'Processing Fold {fold+1}/{k}')

        # Initialize band scores separately for CCACSP and CSP
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))

        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data
            t_train = data_tongue[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_tongue[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            t_val = data_tongue[subject, band][:,:,valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_tongue[subject, band][:,:,valIdx[fold]].transpose(2,0,1)

            train_data = np.stack((t_train, OVR_train), axis=0)
            val_data = np.stack((t_val, OVR_val), axis=0)

            # CCACSP
            ccacsp_filts, clf_CCACSP = train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf_CCACSP)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf_CCACSP)

            band_score_train_CCACSP[:, band] = train_prob_CCACSP
            band_score_val_CCACSP[:, band] = val_prob_CCACSP

            # CSP 
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf_CSP)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf_CSP)


            band_score_train_CSP[:, band] = train_prob_CSP
            band_score_val_CSP[:, band] = val_prob_CSP


        # True labels
        y_train = np.append(np.ones(train_size), np.zeros(train_size))
        y_val = np.append(np.ones(val_size), np.zeros(val_size))

        # CCACSP metrics
        y_pred_train_CCACSP = np.rint(band_score_train_CCACSP.mean(1))
        y_pred_val_CCACSP = np.rint(band_score_val_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = sum(y_pred_val_CCACSP == y_val)/len(y_val)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_pred_val_CCACSP, y_val)

        # CSP metrics
        y_pred_train_CSP = np.rint(band_score_train_CSP.mean(1))
        y_pred_val_CSP = np.rint(band_score_val_CSP.mean(1))
        accuracy_CSP[subject, fold] = sum(y_pred_val_CSP == y_val)/len(y_val)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_pred_val_CSP, y_val)

    print('===============')

# Final Output
print("CCACSP mean accuracy per subject:", np.mean(accuracy_CCACSP, axis=1))
print("CCACSP mean kappa per subject:", np.mean(kappa_CCACSP, axis=1))
print("CSP mean accuracy per subject:", np.mean(accuracy_CSP, axis=1))
print("CSP mean kappa per subject:", np.mean(kappa_CSP, axis=1))

## Left vs Rest - Test Data

In [ ]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))
# Useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
test_size = 72
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))
        band_score_test_CCACSP = np.zeros((test_size*2, n_filters))
        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))
        band_score_test_CSP = np.zeros((test_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data as in your original script
            l_train = data_left[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_left[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            
            l_val = data_left[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_left[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            
            l_test = test_data_left[subject, band][:,:,:].transpose(2,0,1)
            OVR_test = test_data_OVR_left[subject, band][:,:,:].transpose(2,0,1)

            train_data = np.stack((l_train, OVR_train), axis=0)
            val_data = np.stack((l_val, OVR_val), axis=0)
            test_data = np.stack((l_test, OVR_test), axis=0)

            # Run CCACSP
            ccacsp_filts, clf_CCACSP = train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf)
            test_prob_CCACSP = test(np.vstack(test_data), ccacsp_filts, clf)

            # Run CSP
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf)
            test_prob_CSP = csp_test(np.vstack(test_data), csp_filts, clf)

            # Stack probabilities for each band
            band_score_train_CCACSP[:,band] = train_prob_CCACSP
            band_score_val_CCACSP[:,band] = val_prob_CCACSP
            band_score_test_CCACSP[:, band] = test_prob_CCACSP
            band_score_train_CSP[:,band] = train_prob_CSP
            band_score_val_CSP[:,band] = val_prob_CSP
            band_score_test_CSP[:, band] = test_prob_CSP

        # True labels
        y_true = np.append(np.ones(72), np.zeros(72))

        # CCACSP predictions & accuracy/kappa
        y_pred_CCACSP = np.rint(band_score_test_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = accuracy_score(y_true, y_pred_CCACSP)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CCACSP)

        # CSP predictions and accuracy
        y_pred_CSP = np.rint(band_score_test_CSP.mean(1))
        accuracy_CSP[subject, fold] = accuracy_score(y_true, y_pred_CSP)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CSP)

    print(f'Subject {subject+1} done.')
    print('===============')

# Final Mean Accuracies and Kappas for each method
print("Mean Accuracy (CCACSP):", np.mean(accuracy_CCACSP, axis=1))
print("Mean Kappa (CCACSP):", np.mean(kappa_CCACSP, axis=1))
print("Mean Accuracy (Classic CSP):", np.mean(accuracy_CSP, axis=1))
print("Mean Kappa (CSP):", np.mean(kappa_CSP, axis=1))

## Right vs Rest - Test Data

In [ ]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))
# Useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
test_size = 72
csp_per_class = 3

accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))
        band_score_test_CCACSP = np.zeros((test_size*2, n_filters))
        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))
        band_score_test_CSP = np.zeros((test_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data as in your original script
            r_train = data_right[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_right[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            
            r_val = data_right[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_right[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            
            r_test = test_data_right[subject, band][:,:,:].transpose(2,0,1)
            OVR_test = test_data_OVR_right[subject, band][:,:,:].transpose(2,0,1)

            train_data = np.stack((r_train, OVR_train), axis=0)
            val_data = np.stack((r_val, OVR_val), axis=0)
            test_data = np.stack((r_test, OVR_test), axis=0)

            # Run CCACSP
            ccacsp_filts, clf_CCACSP = train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf)
            test_prob_CCACSP = test(np.vstack(test_data), ccacsp_filts, clf)

            # Run CSP
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf)
            test_prob_CSP = csp_test(np.vstack(test_data), csp_filts, clf)

            # Stack probabilities for each band
            band_score_train_CCACSP[:,band] = train_prob_CCACSP
            band_score_val_CCACSP[:,band] = val_prob_CCACSP
            band_score_test_CCACSP[:, band] = test_prob_CCACSP
            band_score_train_CSP[:,band] = train_prob_CSP
            band_score_val_CSP[:,band] = val_prob_CSP
            band_score_test_CSP[:, band] = test_prob_CSP

        # True labels
        y_true = np.append(np.ones(72), np.zeros(72))

        # CCACSP predictions & accuracy/kappa
        y_pred_CCACSP = np.rint(band_score_test_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = accuracy_score(y_true, y_pred_CCACSP)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CCACSP)

        # CSP predictions and accuracy
        y_pred_CSP = np.rint(band_score_test_CSP.mean(1))
        accuracy_CSP[subject, fold] = accuracy_score(y_true, y_pred_CSP)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CSP)

    print(f'Subject {subject+1} done.')
    print('===============')

# Final Mean Accuracies and Kappas for each method
print("Mean Accuracy (CCACSP):", np.mean(accuracy_CCACSP, axis=1))
print("Mean Kappa (CCACSP):", np.mean(kappa_CCACSP, axis=1))
print("Mean Accuracy (Classic CSP):", np.mean(accuracy_CSP, axis=1))
print("Mean Kappa (CSP):", np.mean(kappa_CSP, axis=1))

## Feet vs Rest - Test Data

In [ ]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))
# Useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
test_size = 72
csp_per_class = 3

accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))
        band_score_test_CCACSP = np.zeros((test_size*2, n_filters))
        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))
        band_score_test_CSP = np.zeros((test_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data as in your original script
            f_train = data_feet[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_feet[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            
            f_val = data_feet[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_feet[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            
            f_test = test_data_feet[subject, band][:,:,:].transpose(2,0,1)
            OVR_test = test_data_OVR_feet[subject, band][:,:,:].transpose(2,0,1)

            train_data = np.stack((f_train, OVR_train), axis=0)
            val_data = np.stack((f_val, OVR_val), axis=0)
            test_data = np.stack((f_test, OVR_test), axis=0)

            # Run CCACSP
            ccacsp_filts, clf_CCACSP = train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf)
            test_prob_CCACSP = test(np.vstack(test_data), ccacsp_filts, clf)

            # Run CSP
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf)
            test_prob_CSP = csp_test(np.vstack(test_data), csp_filts, clf)

            # Stack probabilities for each band
            band_score_train_CCACSP[:,band] = train_prob_CCACSP
            band_score_val_CCACSP[:,band] = val_prob_CCACSP
            band_score_test_CCACSP[:, band] = test_prob_CCACSP
            band_score_train_CSP[:,band] = train_prob_CSP
            band_score_val_CSP[:,band] = val_prob_CSP
            band_score_test_CSP[:, band] = test_prob_CSP

        # True labels
        y_true = np.append(np.ones(72), np.zeros(72))

        # CCACSP predictions & accuracy/kappa
        y_pred_CCACSP = np.rint(band_score_test_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = accuracy_score(y_true, y_pred_CCACSP)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CCACSP)

        # CSP predictions and accuracy
        y_pred_CSP = np.rint(band_score_test_CSP.mean(1))
        accuracy_CSP[subject, fold] = accuracy_score(y_true, y_pred_CSP)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CSP)

    print(f'Subject {subject+1} done.')
    print('===============')

# Final Mean Accuracies and Kappas for each method
print("Mean Accuracy (CCACSP):", np.mean(accuracy_CCACSP, axis=1))
print("Mean Kappa (CCACSP):", np.mean(kappa_CCACSP, axis=1))
print("Mean Accuracy (Classic CSP):", np.mean(accuracy_CSP, axis=1))
print("Mean Kappa (CSP):", np.mean(kappa_CSP, axis=1))

## Tongue vs Rest -Test Data

In [ ]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))
# Useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
test_size = 72
csp_per_class = 3

accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))
        band_score_test_CCACSP = np.zeros((test_size*2, n_filters))
        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))
        band_score_test_CSP = np.zeros((test_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data as in your original script
            t_train = data_tongue[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_tongue[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            
            t_val = data_fet[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_feet[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            
            f_test = test_data_feet[subject, band][:,:,:].transpose(2,0,1)
            OVR_test = test_data_OVR_feet[subject, band][:,:,:].transpose(2,0,1)

            train_data = np.stack((f_train, OVR_train), axis=0)
            val_data = np.stack((f_val, OVR_val), axis=0)
            test_data = np.stack((f_test, OVR_test), axis=0)

            # Run CCACSP
            ccacsp_filts, clf_CCACSP = train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf)
            test_prob_CCACSP = test(np.vstack(test_data), ccacsp_filts, clf)

            # Run CSP
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf)
            test_prob_CSP = csp_test(np.vstack(test_data), csp_filts, clf)

            # Stack probabilities for each band
            band_score_train_CCACSP[:,band] = train_prob_CCACSP
            band_score_val_CCACSP[:,band] = val_prob_CCACSP
            band_score_test_CCACSP[:, band] = test_prob_CCACSP
            band_score_train_CSP[:,band] = train_prob_CSP
            band_score_val_CSP[:,band] = val_prob_CSP
            band_score_test_CSP[:, band] = test_prob_CSP

        # True labels
        y_true = np.append(np.ones(72), np.zeros(72))

        # CCACSP predictions & accuracy/kappa
        y_pred_CCACSP = np.rint(band_score_test_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = accuracy_score(y_true, y_pred_CCACSP)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CCACSP)

        # CSP predictions and accuracy
        y_pred_CSP = np.rint(band_score_test_CSP.mean(1))
        accuracy_CSP[subject, fold] = accuracy_score(y_true, y_pred_CSP)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CSP)

    print(f'Subject {subject+1} done.')
    print('===============')

# Final Mean Accuracies and Kappas for each method
print("Mean Accuracy (CCACSP):", np.mean(accuracy_CCACSP, axis=1))
print("Mean Kappa (CCACSP):", np.mean(kappa_CCACSP, axis=1))
print("Mean Accuracy (Classic CSP):", np.mean(accuracy_CSP, axis=1))
print("Mean Kappa (CSP):", np.mean(kappa_CSP, axis=1))